
# PyTorch

[Documentation](https://docs.pytorch.org/docs/2.5/)

In [4]:
import numpy as np
import torch

In [5]:

print(torch.cuda.is_available())

False


In [6]:


np_array = np.array([1.0, 2.0, 3.0])

# Create Tensor from Numpy Array
tensor = torch.from_numpy(np_array)

# print(type(np_array))  # <class 'numpy.ndarray'>
# print(type(tensor))    # <class 'torch.Tensor'>

# tensor = tensor.to('cuda')
# print(type(tensor)) # <class 'torch.Tensor'>


In [7]:
# convert to numpy 
np_array_back = tensor.numpy()
print(type(np_array_back))

<class 'numpy.ndarray'>


## 余弦相似度


In [8]:
def cosine_similarity(v1, v2):
    dot = torch.dot(v1, v2)
    norm_v1 = torch.norm(v1) 
    norm_v2 = torch.norm(v2)
    return dot / (norm_v1 * norm_v2) 

# 测试数据（模拟两个文本的向量）
tensor_a = torch.tensor([1.2, 0.5, 3.1])
tensor_b = torch.tensor([1.1, 0.6, 3.0]) # 这个应该和 a 很像
tensor_c = torch.tensor([-1.0, 2.0, 0.1]) # 这个应该差别很大

print(f"A和B的相似度: {cosine_similarity(tensor_a, tensor_b)}")
print(f"A和C的相似度: {cosine_similarity(tensor_a, tensor_c)}")

A和B的相似度: 0.9991849660873413
A和C的相似度: 0.014619564637541771


## unsqueeze、squeeze

escalate

In [9]:
data = torch.randn(512, 768)
data.shape

torch.Size([512, 768])

In [10]:
input_to_model = data.unsqueeze(0)
input_to_model.shape

torch.Size([1, 512, 768])

descend

In [11]:
back_to_original = input_to_model.squeeze(0)
back_to_original.shape

torch.Size([512, 768])

In [12]:
x = torch.zeros(2, 1, 2, 1, 2)
x.size()
torch.Size([2, 1, 2, 1, 2])
y = torch.squeeze(x)
y.size()
# squeeze 会删除所有 size == 1 的维度

torch.Size([2, 2, 2])

In [13]:
repeat_model = input_to_model.repeat(2, 1, 1)
repeat_model.shape

torch.Size([2, 512, 768])

In [14]:
# × 不能删除所有 size != 1 的维度
repeat_model.squeeze(0).shape

torch.Size([2, 512, 768])

Q: PyTorc和numpy都可以做升降维度，应该选择使用哪个？

| 场景                         | 建议              |
| -------------------------- | --------------- |
| 数据马上要送进 **PyTorch 模型**     | **用 `torch`**   |
| 数据只是做 **前处理 / 后处理 / 数值计算** | **用 `numpy`**   |
| 已经是 `torch.Tensor`         | **不要再转回 NumPy** |
| 需要 GPU / 自动求导              | **必须用 `torch`** |


## Common operation in Numpy and PyTorch

| 操作语义      | NumPy                | PyTorch          |
| --------- | -------------------- | ---------------- |
| 加 batch 维 | `x[np.newaxis, ...]` | `x.unsqueeze(0)` |
| 去 batch 维 | `np.squeeze(x, 0)`   | `x.squeeze(0)`   |
| 查看维度      | `x.ndim`             | `x.dim()`        |
| 查看 shape  | `x.shape`            | `x.shape`        |
| 转置        | `np.transpose`       | `x.permute`      |


## reshape

In [15]:
# 常见场景： 把 [Batch, Sequence, Embedding] 压扁成 [Batch * Sequence, Embedding] 给全连接层处理。
# 假设这是 Transformer 的输出 [Batch=2, Seq=4, Dim=8]
batch_output = torch.randn(2, 4, 8)
batch_output.shape

torch.Size([2, 4, 8])

In [16]:
# 场景：我们要把这 8 个数字分类，需要把前两个维度合并
# 变成 [8, 8]，即 [2*4, 8]
flattened = batch_output.reshape(-1, 8)

# -1 的意思是：PyTorch 你自己算这维应该是多少 (2*4=8)
flattened.shape

torch.Size([8, 8])

In [17]:
# reverse the action
back_to_before_reshaped = flattened.reshape(2, 4, 8)
back_to_before_reshaped.shape

torch.Size([2, 4, 8])

In [18]:
# 变成1维
flattened.flatten().shape

torch.Size([64])

## transpose and permute

`transpose` 只能交换 **两个** 维度。

`permute` 可以重新排列 **所有** 维度（更强，推荐用这个）。

常见场景： 图像处理习惯用 [Channel, Height, Width]，但有的库（如 matplotlib）画图需要 [Height, Width, Channel]。

In [19]:
image_tensor = torch.randn(3, 256, 256)
image_tensor.shape

torch.Size([3, 256, 256])

In [20]:
image_plot = image_tensor.permute(1, 2, 0)
image_plot.shape

torch.Size([256, 256, 3])

## autograd

In [21]:
import torch

def understand_autograd():
    # 1. 创建一个模拟的模型参数 (需要求导)
    # create a weight tensor with requires_grad=True
    w = torch.randn(10, requires_grad=True) 
    x = torch.randn(10)
    
    print(f"初始 w 的梯度: {w.grad}") # None
    
    # --- 场景 A: 训练模式 (默认) ---
    loss = (w * x).sum() # 这是一个简单的计算
    loss.backward()      # 反向传播，计算梯度
    
    print(f"训练模式下，反向传播后 w 的梯度: \n{w.grad}") 
    # 你会看到一堆数字，这就是模型“学习”的依据
    
    # --- 场景 B: 推理模式 (Inference) ---
    # 💥这是你未来写 RAG 代码时 100% 会用到的
    print("\n=== 进入推理模式 (torch.no_grad) ===")
    
    # 清空之前的梯度 (PyTorch 会累加梯度，所以每次要清零)
    w.grad = None 
    
    with torch.no_grad():
        # 在这缩进块里的所有计算，都不会记录梯度
        # 显存占用会大幅下降，速度变快
        loss_inference = (w * x).sum()
        
        # loss_inference.backward() 
        # 如果你取消上面这行的注释，会报错！因为没有计算图。
        
    print("在 torch.no_grad() 中计算完成。")
    print(f"推理模式下，w 的梯度是否变化? {w.grad}") # 应该是 None

understand_autograd()

初始 w 的梯度: None
训练模式下，反向传播后 w 的梯度: 
tensor([-0.5480, -0.7442,  0.5113, -1.1995,  0.4100,  0.0666,  0.1681, -1.0720,
        -0.1468, -0.9393])

=== 进入推理模式 (torch.no_grad) ===
在 torch.no_grad() 中计算完成。
推理模式下，w 的梯度是否变化? None


## DataSet

In [ ]:
from torch.utils.data import Dataset
import torch

In [61]:
# 假设这是你的原始数据（实际场景中可能是一个 CSV 文件路径）
raw_data = [
    ("我想退货", 1),       # 1 表示负面
    ("这个产品太棒了", 0), # 0 表示正面
    ("快递太慢了", 1),
    ("客服态度很好", 0),
    ("质量一般般", 1),
    ("下次还来买", 0)
]

class MyTextDataset(Dataset):
    def __init__(self, data_source):
        self.data = data_source

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        """
        核心！告诉 PyTorch 第 idx 个数据怎么处理
        实际项目中，这里会包含：
        1. 读取图片/文本
        2. 预处理 (Resize, Tokenize)
        3. 转成 Tensor
        """
        text, label = self.data[index]
        feature_vector = torch.tensor([len(text)], dtype=torch.float32)
        label_vector = torch.tensor(label, dtype=torch.long)
        return feature_vector, label_vector

print(raw_data)
my_dataset = MyTextDataset(raw_data)

print(f"total length of data {len(my_dataset)}")
print(f"the second item: {my_dataset[2]}")

[('我想退货', 1), ('这个产品太棒了', 0), ('快递太慢了', 1), ('客服态度很好', 0), ('质量一般般', 1), ('下次还来买', 0)]
total length of data 6
the second item: (tensor([5.]), tensor(1))


## DataLoader

In [ ]:
from torch.utils.data import DataLoader


data_loader = DataLoader(dataset=my_dataset, batch_size=2, shuffle=True, drop_last=True)

for epoch in range(1):
    print(f"\nEpoch {epoch+1}:")
    for step, (batch_features, batch_labels) in enumerate(data_loader):
        print(f"  Step {step}:")
        '''
            2 size of the batch size, 1 original 
           输入 Batch 形状: torch.Size([2, 1]) 
           标签 Batch 数据: tensor([1, 0])

            feature_vector = torch.tensor([len(text)], dtype=torch.float32) # 一维向量
            label_vector = torch.tensor(label, dtype=torch.long)  # 标量
        '''
        print(f"    输入 Batch 数据: {batch_features}") # tensor([[7.],[5.]])
        print(f"    输入 Batch 形状: {batch_features.shape}") # [2, 1] 2d
        print(f"    标签 Batch 数据: {batch_labels}") # tensor([1, 0])
        print(f"    标签 Batch 形状: {batch_labels.shape}") # torch.Size([2]) 1d


Epoch 1:
  Step 0:
    输入 Batch 数据: tensor([[4.],
        [5.]])
    输入 Batch 形状: torch.Size([2, 1])
    标签 Batch 数据: tensor([1, 0])
    标签 Batch 形状: torch.Size([2])
  Step 1:
    输入 Batch 数据: tensor([[5.],
        [5.]])
    输入 Batch 形状: torch.Size([2, 1])
    标签 Batch 数据: tensor([1, 1])
    标签 Batch 形状: torch.Size([2])
  Step 2:
    输入 Batch 数据: tensor([[6.],
        [7.]])
    输入 Batch 形状: torch.Size([2, 1])
    标签 Batch 数据: tensor([0, 0])
    标签 Batch 形状: torch.Size([2])


----

| 项目        | feature_vector      | label_vector      |
| --------- | ------------------- | ----------------- |
| 创建方式      | `torch.tensor([x])` | `torch.tensor(x)` |
| Python 输入 | list                | 标量                |
| Tensor 维度 | `[1]`               | `[]`（0 维）         |
| batch 后   | `[B, 1]`            | `[B]`             |
| 工程语义      | 输入特征                | 监督标签              |
| 常见 dtype  | `float32`           | `long`            |
| 是否参与反向传播  | 是                   | 否                 |


----

PyTorch 训练中的隐含契约

| 角色                         | 期望 shape           |
| -------------------------- | ------------------ |
| Dataset → feature          | `[feature_dim]`    |
| DataLoader → batch feature | `[B, feature_dim]` |
| Dataset → label            | `[]`               |
| DataLoader → batch label   | `[B]`              |
| Loss 函数                    | `[B]`              |
